#### **Задача:** ####

Рассчитать среднее время ответа для каждого менеджера/пары менеджеров.

Расчёт должен учитывать следующее: 
* если в диалоге идут несколько сообщений подряд от клиента или менеджера, то при расчёте времени ответа надо учитывать только первое сообщение из каждого блока; 
* менеджеры работают с 09:30 до 00:00, поэтому нерабочее время не должно учитываться в расчёте среднего времени ответа, т.е. если клиент написал в 23:59, а менеджер ответил в 09:30 – время ответа равно одной минуте; 
* ответы на сообщения, пришедшие ночью также нужно учитывать. 

**Загрузка данных из таблиц в датафреймы**

In [1]:
# Импорт библиотек
from sqlalchemy import create_engine    # для рпботы с базой данных
import urllib.parse                     # для кодирования пароля
import pandas as pd                     # для обработки табличных данных
from datetime import datetime, time, timedelta     # для работы со временем
import pytz                             # для временных зон

# Исходные данные для подключения
host="rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net"
database="db1"
user="test_user"
password="j2M{CnnFq@"
port=6432

# Кодирование пароля
encoded_password = urllib.parse.quote(password)

# Формирование строки подключения
connection_string = f'postgresql+psycopg2://{user}:{encoded_password}@{host}:{port}/{database}'

# Создание движка SQLAlchemy
engine = create_engine(connection_string)

# Загрузка данных из таблиц в датафреймы
chat_messages_df = pd.read_sql_query("SELECT * FROM test.chat_messages", engine)
managers_df = pd.read_sql_query("SELECT * FROM test.managers", engine)

# Принудительная остановка движка
engine.dispose()

# Вывод таблиц
display(chat_messages_df.head(2))
display(managers_df.head(2))

,message_id,type,entity_id,created_by,created_at
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924
1,"""01jb3t6bc8gvyfnd063nd91ng4""",incoming_chat_message,37549491,0,1729926213


,mop_id,name_mop,rop_id
0,6645315,Гюнель и Илина,1
1,6744792,Юля и Наташа,1


#### **Преобразование данных**

Избавимся от очереди сообщений от одного источника

In [2]:
# Приводим время в формат datetime с учетом временной зоны UTC+3
chat_messages_df['created_at'] = (
    pd.to_datetime(chat_messages_df['created_at'], unit='s', utc=True)
    .dt.tz_convert('Europe/Moscow')
    )

# Используем SHIFT (аналог LAG в sql) для определения начала нового блока сообщений
chat_messages_df = chat_messages_df.sort_values(by=['entity_id', 'created_at'])
chat_messages_df['prev_type'] = chat_messages_df.groupby('entity_id')['type'].shift()

# Оставляем только первые сообщения из каждого блока
# null - первое сообщение в сделке
# prev_type отличается от текущего type
messages_mask = chat_messages_df['prev_type'].isna() | \
    (chat_messages_df['prev_type'] != chat_messages_df['type'])
filtered_messages_df = chat_messages_df[messages_mask]

filtered_messages_df.drop('prev_type', axis=1, inplace=True)
display(filtered_messages_df.head(4))

,message_id,type,entity_id,created_by,created_at
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,2024-10-26 18:19:26+03:00
4102,"""01jb4pncn7gm1re1zdxkzhnp6s""",outgoing_chat_message,35810509,10262493,2024-10-26 18:21:05+03:00
6430,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,2024-10-26 18:34:46+03:00
6639,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,2024-10-26 18:36:46+03:00


Добавим к каждому сообщению клиенту время ответа менеджера

In [3]:
# Разделяем сообщения клиента и менеджера
client_messages_df = filtered_messages_df[filtered_messages_df['type'] == 'incoming_chat_message']
manager_messages_df = filtered_messages_df[filtered_messages_df['type'] == 'outgoing_chat_message']

# Объединяем сообщения клиента и менеджера
merged_df = pd.merge(
    client_messages_df,
    manager_messages_df,
    on='entity_id',
    suffixes=('_client', '_manager'),
    how='inner'
)

# Фильтруем только те строки, где время ответа менеджера больше времени сообщения клиента
filtered_df = merged_df[merged_df['created_at_manager'] > merged_df['created_at_client']]

# Найдем первый ответ менеджера для каждого сообщения клиента
response_times_df = (
    filtered_df
    .sort_values(['entity_id', 'created_at_client', 'created_at_manager'])
    .groupby(['entity_id', 'created_at_client'])
    .first()
    .reset_index()
)

# Переименуем столбцы для удобства
response_times_df = response_times_df.rename(columns={
    'message_id_client': 'message_id',
    'created_at_client': 'client_message_time',
    'created_at_manager': 'manager_response_time',
    'created_by_manager': 'manager_id'
})

# Оставим необходимые столбцы
required_columns = ['message_id', 'entity_id', 'manager_id',
                    'client_message_time', 'manager_response_time']
response_times_df = response_times_df[required_columns]

display(response_times_df.head())

,message_id,entity_id,manager_id,client_message_time,manager_response_time
0,"""01jb4pjb1g22ts7z6arcjv11w6""",35810509,10262493,2024-10-26 18:19:26+03:00,2024-10-26 18:21:05+03:00
1,"""01jb4qedfgxnd2yqr4w26hs4q5""",35810509,10262493,2024-10-26 18:34:46+03:00,2024-10-26 18:36:46+03:00
2,"""01jb7gxwb0sepv9vtekrgj3v6d""",35945535,10262493,2024-10-27 20:38:36+03:00,2024-10-27 20:42:07+03:00
3,"""01jb727x90ga4ncv4m705rh0rm""",36099337,11241942,2024-10-27 16:21:56+03:00,2024-10-27 16:25:47+03:00
4,"""01jb72hktgm0tvdpwk4a17dadz""",36099337,11241942,2024-10-27 16:27:14+03:00,2024-10-27 16:29:37+03:00


In [4]:
# Корректируем время ответа с учётом рабочего времени
def adjust_response_time(row):
    client_message_date = row['client_message_time'].date()

    # Создаем объекты времени начала и конца рабочего дня с учетом временной зоны
    start_work_time = datetime.combine(client_message_date, time(9, 30)).astimezone(pytz.timezone('Europe/Moscow'))
    end_work_time = datetime.combine(client_message_date + timedelta(days=1), time(0, 0, 0)).astimezone(pytz.timezone('Europe/Moscow'))

    # Если ответ пришёл в рабочее время
    if (row['manager_response_time'].time() >= time(9, 30) and 
        row['client_message_time'].time() >= time(9, 30) and 
        row['manager_response_time'].time() <= time(23, 59, 59) and 
        row['client_message_time'].time() <= time(23, 59, 59)):
        # Считаем разницу в секундах
        return (row['manager_response_time'] - row['client_message_time']).total_seconds()
    
    # Если клиент написал в нерабочее время, а ответ пришёл в рабочее
    elif (row['client_message_time'].time() < time(9, 30) and
          row['manager_response_time'].time() >= time(9, 30)):
        # Принимаем, что клиент написал в начале рабочего дня, и считаем разницу в секундах
        return (row['manager_response_time'] - start_work_time).total_seconds()
    
    # Если клиент написал в рабочее время, а ответ пришёл в нерабочее
    elif (row['client_message_time'].time() >= time(9, 30) and
          row['manager_response_time'].time() > time(23, 59, 59)):
        # Принимаем, что менеджер ответил в конце рабочего дня, и считаем разницу в секундах
        return (end_work_time - row['client_message_time']).total_seconds()
    
    else:
        return 0

response_times_df['adjusted_response_time'] = response_times_df.apply(adjust_response_time, axis=1)

# Объединяем с таблицей менеджеров для получения имени менеджера
response_times_df = response_times_df.merge(
    managers_df,
    left_on='manager_id',
    right_on='mop_id',
    how='left'
    )

display(response_times_df.head())

,message_id,entity_id,manager_id,client_message_time,manager_response_time,adjusted_response_time,mop_id,name_mop,rop_id
0,"""01jb4pjb1g22ts7z6arcjv11w6""",35810509,10262493,2024-10-26 18:19:26+03:00,2024-10-26 18:21:05+03:00,99.0,10262493.0,Ками и Мила,3
1,"""01jb4qedfgxnd2yqr4w26hs4q5""",35810509,10262493,2024-10-26 18:34:46+03:00,2024-10-26 18:36:46+03:00,120.0,10262493.0,Ками и Мила,3
2,"""01jb7gxwb0sepv9vtekrgj3v6d""",35945535,10262493,2024-10-27 20:38:36+03:00,2024-10-27 20:42:07+03:00,211.0,10262493.0,Ками и Мила,3
3,"""01jb727x90ga4ncv4m705rh0rm""",36099337,11241942,2024-10-27 16:21:56+03:00,2024-10-27 16:25:47+03:00,231.0,11241942.0,Даша и Карина,3
4,"""01jb72hktgm0tvdpwk4a17dadz""",36099337,11241942,2024-10-27 16:27:14+03:00,2024-10-27 16:29:37+03:00,143.0,11241942.0,Даша и Карина,3


Рассчитаем среднее время ответа менеджера

In [5]:
avg_response_time = response_times_df.groupby('name_mop')['adjusted_response_time'].mean().reset_index()
avg_response_time['avg_response_time_seconds'] = avg_response_time['adjusted_response_time'].round().astype(int)
avg_response_time['avg_response_time'] = pd.to_timedelta(avg_response_time['avg_response_time_seconds'], unit='s')
avg_response_time = avg_response_time[['name_mop', 'avg_response_time_seconds', 'avg_response_time']]
display(avg_response_time.head(30))

,name_mop,avg_response_time_seconds,avg_response_time
0,Алина и Юля,273,0 days 00:04:33
1,Ангелина Милованова,366,0 days 00:06:06
2,Аня и Ксюша,817,0 days 00:13:37
3,Вика и Катя,798,0 days 00:13:18
4,Влада и Настя,283,0 days 00:04:43
5,Гюнель и Илина,513,0 days 00:08:33
6,Даша и Влада,978,0 days 00:16:18
7,Даша и Даша,237,0 days 00:03:57
8,Даша и Карина,455,0 days 00:07:35
9,Ира и Варя,1415,0 days 00:23:35


**Постскриптум:**

В боевой задаче не следует создавать столько объектов DataFarame.

Здесь я это сделал для лучшей наглядности, и последовательности вычислений.